In [4]:
!pip install selenium
!pip install bs4
!pip install webdriver-manager
!pip install pandas
!pip install lxml


     ---------------------------------------- 11.0/11.0 MB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 14.7/14.7 MB 1.6 MB/s eta 0:00:00

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pysd 3.0.1 requires chardet, which is not installed.
pysd 3.0.1 requires openpyxl, which is not installed.
pysd 3.0.1 requires parsimonious, which is not installed.
pysd 3.0.1 requires progressbar2, which is not installed.
pysd 3.0.1 requires regex, which is not installed.
pysd 3.0.1 requires scipy, which is not installed.
pysd 3.0.1 requires xarray, which is not installed.
pysd 3.0.1 requires xlrd, which is not installed.


In [6]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://apps5.mineco.gob.pe/transparencia/Navegador/Navegar_7.aspx")

# to click on input field
driver.find_element(By.XPATH,"//*[@id='ctl00_CPH1_BtnTipoGobierno']").click()

time.sleep(5)

# to click on the element(E: GOBIERNO NACIONAL) found
driver.find_element(By.XPATH,"//*[@id='ctl00_CPH1_RptData_ctl01_TD0']").click()

time.sleep(5)

# to click on input field
driver.find_element(By.XPATH,"//*[@id='ctl00_CPH1_BtnSector']").click()

time.sleep(5)

page_source = driver.page_source
print(page_source)

driver.close()


<html xmlns="http://www.w3.org/1999/xhtml"><head><link href="../App_Themes/Tema1/StyleSearch.css" type="text/css" rel="stylesheet"><link href="../App_Themes/Tema1/StyleSheet.css" type="text/css" rel="stylesheet">
    <!-- Global site tag (gtag.js) - Google Analytics -->
    <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-V4ESNG1Y68"></script>
    <script>
      window.dataLayer = window.dataLayer || [];
      function gtag(){dataLayer.push(arguments);}
      gtag('js', new Date());

      gtag('config', 'G-V4ESNG1Y68');
    </script>
    <script language="javascript" type="text/javascript">
	<!--
	function OnClickTab(tb)
	{
		var params = GetUrlParams();
		window.parent.location = "default.aspx?tb=" + tb + params;
	}	

	function OnSubmit()
	{
		if (onSubmitValid)
			return true;
		else
		{
			onSubmitValid = true;
			return false;
		}
	}
	//-->
	</script>
<title>
	Consulta Amigable - Navegador
</title></head>
<body onload="if (this.OnLoad) OnLoad();">
    <form meth

In [7]:
soup = BeautifulSoup(page_source, 'lxml')
sectors_useful_data = []
table_data = soup.select(".Data")
sectors_selector = table_data[0].find_all('tr')
for sector in sectors_selector:
    sector_columns = sector.find_all("td")
    sector_info = []
    sector_info.append(sector_columns[1].string.strip())
    sector_info.append(sector_columns[3].string.strip())
    sector_info.append(sector_columns[7].string.strip())
    sector_info.append(sector_columns[9].string.strip())
    sectors_useful_data.append(sector_info)
print(sectors_useful_data)

[['01: PRESIDENCIA CONSEJO MINISTROS', '5,730,751,623', '4,642,398,604'], ['03: CULTURA', '605,268,276', '459,873,549'], ['04: PODER JUDICIAL', '3,196,787,232', '2,597,676,003'], ['05: AMBIENTAL', '758,694,437', '552,253,113'], ['06: JUSTICIA', '2,143,538,893', '1,713,025,696'], ['07: INTERIOR', '11,974,940,979', '9,950,852,728'], ['08: RELACIONES EXTERIORES', '952,058,119', '799,592,259'], ['09: ECONOMIA Y FINANZAS', '37,684,687,384', '28,581,070,519'], ['10: EDUCACION', '14,609,012,977', '9,687,999,959'], ['11: SALUD', '12,917,327,130', '9,874,805,703'], ['12: TRABAJO Y PROMOCION DEL EMPLEO', '1,342,345,369', '1,237,961,223'], ['13: AGRARIO Y DE RIEGO', '3,332,543,185', '2,391,475,771'], ['16: ENERGIA Y MINAS', '1,423,632,099', '1,089,363,340'], ['19: CONTRALORIA GENERAL', '915,005,464', '649,524,946'], ['20: DEFENSORIA DEL PUEBLO', '73,760,872', '60,348,956'], ['21: JUNTA NACIONAL DE JUSTICIA', '41,722,957', '33,311,937'], ['22: MINISTERIO PUBLICO', '2,691,682,758', '2,248,555,766']

In [27]:
import pandas as pd

df = pd.read_csv('./consulta-amigable.csv', 
            index_col=False, 
            header=0, 
            names=['Sector', 'PIM', 'Devengado', 'Avance'])
print(df)
print('After')
df.loc[0,['Sector', 'PIM', 'Devengado', 'Avance']] = ['Sector', 'PIM', 'Devengado', 'Avance']
print(df)
df.to_csv('./consulta-amigable.csv',index=False)

   Sector      PIM  Devengado     Avance
0  Sector      PIM  Devengado     Avance
1       0  Sectoro        PIM  Devengado
After
   Sector      PIM  Devengado     Avance
0  Sector      PIM  Devengado     Avance
1       0  Sectoro        PIM  Devengado
